<a href="https://colab.research.google.com/github/noahflore/colab/blob/main/atividade2_analise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analisando o dataset de celulares. Neste exemplo, vou usar o pandas para entender do que se trata o conteúdo sem usar referências externas.
## O arquivo foi baixado do Kaggle.

In [ ]:
# Identificando a codificação do arquivo CSV
# O arquivo foi baixado do Kaggle e possui codificação diferente do UTF-8
with open("/content/Mobiles Dataset (2025).csv", 'rb') as f:
    result = chardet.detect(f.read())
print(result['encoding'])

# Lendo o arquivo CSV com a codificação detectada
df = pd.read_csv("/content/Mobiles Dataset (2025).csv", encoding=result['encoding'])

ISO-8859-1


Foi identificado que o arquivo possui uma codificação diferente do UTF-8.

In [ ]:
# Análise inicial do dataset
print(df.head())  # Exibe as primeiras linhas do dataframe
print("\n---------------------------------")
print(df.describe())  # Estatísticas descritivas das colunas numéricas
print("\n---------------------------------")
print(df.info())  # Informações sobre tipos de dados e valores não nulos

  Company Name            Model Name Mobile Weight  RAM Front Camera  \
0        Apple       iPhone 16 128GB          174g  6GB         12MP   
1        Apple       iPhone 16 256GB          174g  6GB         12MP   
2        Apple       iPhone 16 512GB          174g  6GB         12MP   
3        Apple  iPhone 16 Plus 128GB          203g  6GB         12MP   
4        Apple  iPhone 16 Plus 256GB          203g  6GB         12MP   

  Back Camera   Processor Battery Capacity Screen Size  \
0        48MP  A17 Bionic         3,600mAh  6.1 inches   
1        48MP  A17 Bionic         3,600mAh  6.1 inches   
2        48MP  A17 Bionic         3,600mAh  6.1 inches   
3        48MP  A17 Bionic         4,200mAh  6.7 inches   
4        48MP  A17 Bionic         4,200mAh  6.7 inches   

  Launched Price (Pakistan) Launched Price (India) Launched Price (China)  \
0               PKR 224,999             INR 79,999              CNY 5,799   
1               PKR 234,999             INR 84,999              

Pelo resultado, podemos analisar que se trata de uma lista de smartphones lançados em 2024. Podemos realizar uma análise mais profunda, porém, devemos verificar a qualidade dos dados, checando se há valores nulos, redundantes ou faltantes.

In [ ]:
# Verificação de valores nulos no dataset
print(df.isnull().sum())  # Quantidade de valores nulos por coluna
print(df.isnull().mean() * 100)  # Porcentagem de valores nulos por coluna

Company Name                 0
Model Name                   0
Mobile Weight                0
RAM                          0
Front Camera                 0
Back Camera                  0
Processor                    0
Battery Capacity             0
Screen Size                  0
Launched Price (Pakistan)    0
Launched Price (India)       0
Launched Price (China)       0
Launched Price (USA)         0
Launched Price (Dubai)       0
Launched Year                0
dtype: int64
Company Name                 0.0
Model Name                   0.0
Mobile Weight                0.0
RAM                          0.0
Front Camera                 0.0
Back Camera                  0.0
Processor                    0.0
Battery Capacity             0.0
Screen Size                  0.0
Launched Price (Pakistan)    0.0
Launched Price (India)       0.0
Launched Price (China)       0.0
Launched Price (USA)         0.0
Launched Price (Dubai)       0.0
Launched Year                0.0
dtype: float64


In [ ]:
# Verificação de linhas duplicadas
print(df.duplicated().sum())  # Quantidade de linhas duplicadas
print("\n-------------------------")
print(df[df.duplicated(keep=False)])  # Exibe todas as linhas duplicadas

15

-------------------------
    Company Name        Model Name Mobile Weight   RAM Front Camera  \
344         Oppo          A3 128GB          186g   4GB          5MP   
347         Oppo          A3 128GB          186g   4GB          5MP   
423         Oppo        K11x 128GB          195g   8GB         16MP   
424         Oppo        K11x 256GB          195g  12GB         16MP   
425         Oppo        K10x 128GB          195g   6GB         16MP   
426         Oppo        K10x 256GB          195g   8GB         16MP   
427         Oppo      K10 5G 128GB          205g   8GB         16MP   
428         Oppo         K9x 128GB          194g   6GB         16MP   
429         Oppo         K9x 256GB          194g   8GB         16MP   
430         Oppo   K9 Pro 5G 128GB          180g   8GB         16MP   
431         Oppo   K9 Pro 5G 256GB          180g  12GB         16MP   
432         Oppo       K9 5G 128GB          172g   8GB         32MP   
433         Oppo       K9 5G 256GB          172

In [ ]:
# Mostra os valores únicos de uma coluna (útil para categóricos)
# print(df['coluna'].unique())

#como são dados de hardware os valores costuma se duplicados então provavelmente não existe únicos


agora iremos fazer uma filtragem em 3 colunas para analise

In [ ]:
# Limpeza e conversão das colunas de preço para análise
def clean_price_column(column):
    return (
        column.astype(str)
        .str.replace(r'[^\d.]', '', regex=True)  # Remove caracteres não numéricos
        .astype(float)  # Converte para float
    )

# Aplicando a limpeza nas colunas de preço
df["Launched Price (USA)"] = clean_price_column(df["Launched Price (USA)"])
df["Launched Price (India)"] = clean_price_column(df["Launched Price (India)"])
df["Launched Price (China)"] = clean_price_column(df["Launched Price (China)"])

# Filtrando smartphones com preços acima de US$500 ou equivalentes
df_filtered = df[
    (df["Launched Price (USA)"] > 500) |
    (df["Launched Price (India)"] > 42642.54) |  # Equivalente a ~US$500
    (df["Launched Price (China)"] > 3643.5)  # Equivalente a ~US$500
]

# Exibindo os resultados filtrados
print(df_filtered.head())

  Company Name            Model Name Mobile Weight  RAM Front Camera  \
0        Apple       iPhone 16 128GB          174g  6GB         12MP   
1        Apple       iPhone 16 256GB          174g  6GB         12MP   
2        Apple       iPhone 16 512GB          174g  6GB         12MP   
3        Apple  iPhone 16 Plus 128GB          203g  6GB         12MP   
4        Apple  iPhone 16 Plus 256GB          203g  6GB         12MP   

  Back Camera   Processor Battery Capacity Screen Size  \
0        48MP  A17 Bionic         3,600mAh  6.1 inches   
1        48MP  A17 Bionic         3,600mAh  6.1 inches   
2        48MP  A17 Bionic         3,600mAh  6.1 inches   
3        48MP  A17 Bionic         4,200mAh  6.7 inches   
4        48MP  A17 Bionic         4,200mAh  6.7 inches   

  Launched Price (Pakistan)  Launched Price (India)  Launched Price (China)  \
0               PKR 224,999                 79999.0                  5799.0   
1               PKR 234,999                 84999.0         

Pelo resultado obtido, os iPhones vendidos nos EUA em 2024 por mais de 500 dólares apresentam preços semelhantes aos de outros países, considerando que utilizei um conversor externo de moeda para fazer a comparação.

In [ ]:
# Análise específica dos modelos iPhone 16
iphones = df[df["Model Name"].str.contains("iPhone 16")]
print(iphones[["Model Name", "Launched Price (USA)", "Launched Price (India)", "Launched Price (China)"]])

                 Model Name  Launched Price (USA)  Launched Price (India)  \
0           iPhone 16 128GB                 799.0                 79999.0   
1           iPhone 16 256GB                 849.0                 84999.0   
2           iPhone 16 512GB                 899.0                 89999.0   
3      iPhone 16 Plus 128GB                 899.0                 89999.0   
4      iPhone 16 Plus 256GB                 949.0                 94999.0   
5      iPhone 16 Plus 512GB                 999.0                104999.0   
6       iPhone 16 Pro 128GB                 999.0                 99999.0   
7       iPhone 16 Pro 256GB                1049.0                104999.0   
8       iPhone 16 Pro 512GB                1099.0                114999.0   
9   iPhone 16 Pro Max 128GB                1099.0                109999.0   
10  iPhone 16 Pro Max 256GB                1199.0                114999.0   
11  iPhone 16 Pro Max 512GB                1299.0                124999.0   

Após uma segunda análise mais robusta, aparenta que, na Índia, os preços são aproximadamente 20% mais altos em comparação aos dos EUA. Agora, vamos analisar essa porcentagem com mais detalhe.

In [ ]:
# Cálculo de diferenças de preço entre países
# Conversão: 1 USD = 83 INR (Rúpia Indiana) e 1 USD = 7.2 CNY (Yuan Chinês)
df['India Premium (%)'] = ((df['Launched Price (India)']/83 - df['Launched Price (USA)'])/df['Launched Price (USA)']*100).round(1)
df['China Discount (%)'] = ((df['Launched Price (USA)'] - df['Launched Price (China)']/7.2)/df['Launched Price (USA)']*100).round(1)

# Exibindo comparação de preços
print(df[['Model Name', 'Launched Price (USA)', 'India Premium (%)', 'China Discount (%)']])

               Model Name  Launched Price (USA)  India Premium (%)  \
0         iPhone 16 128GB                 799.0               20.6   
1         iPhone 16 256GB                 849.0               20.6   
2         iPhone 16 512GB                 899.0               20.6   
3    iPhone 16 Plus 128GB                 899.0               20.6   
4    iPhone 16 Plus 256GB                 949.0               20.6   
..                    ...                   ...                ...   
925          Pad 5G 128GB                 280.0                3.3   
926          Pad 5G 256GB                 300.0                4.4   
927  Galaxy Z Fold6 256GB                1899.0                4.7   
928  Galaxy Z Fold6 512GB                1719.0               24.1   
929    Galaxy Z Fold6 1TB                2259.0                7.2   

     China Discount (%)  
0                  -0.8  
1                   0.2  
2                  -0.4  
3                   4.2  
4                   4.9  
.. 

    Essa última análise leva a crer que, na China, os preços são equivalentes ou até inferiores, enquanto na Índia estão acima, em comparação aos dos EUA.